In [5]:
import numpy as np
import os
import glob
from sklearn.decomposition import IncrementalPCA, PCA
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import multiprocessing  # <--- Importado para paralelismo

In [6]:
# Define input and output directories
in_directory = "./../../hyspecnet-11k/hyspecnet-11k/patches/"
out_directory = "./../../hyspecnet11k-PCA/"
npy_files = glob.glob(f"{in_directory}**/**/*DATA.npy")

n_files = len(npy_files)


# create out directory if dont exist
if not os.path.exists(out_directory):
    os.makedirs(out_directory)

In [7]:
npy_files.sort()
npy_files 

['./../../hyspecnet-11k/hyspecnet-11k/patches/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z-Y01460273_X03110438/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z-Y01460273_X03110438-DATA.npy',
 './../../hyspecnet-11k/hyspecnet-11k/patches/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z-Y01460273_X04390566/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z-Y01460273_X04390566-DATA.npy',
 './../../hyspecnet-11k/hyspecnet-11k/patches/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z-Y01460273_X05670694/ENMAP01-____L2A-DT0000004950_20221103T162438Z_001_V010110_20221118T145147Z-Y01460273_X05670694-DATA.npy',
 './../../hyspecnet-11k/hyspecnet-

In [8]:

# Parâmetros do Group-wise PCA
TOTAL_NC = 32          # Número total de componentes principais desejado no final
NUM_GROUPS = 4           # Em quantos grupos as bandas espectrais serão divididas

# Dimensões esperadas de cada patch (bandas, altura, largura)
DIMS = (202, 128,128)

BATCH_SIZE = 128
# --- Verificações Iniciais ---
if TOTAL_NC % NUM_GROUPS != 0:
    raise ValueError("O número total de componentes (TOTAL_NC) deve ser divisível pelo número de grupos (NUM_GROUPS).")
NC_PER_GROUP = TOTAL_NC // NUM_GROUPS





In [21]:
# Função para processar um único arquivo .npy
def process_single_file(file_path):
    try:
        X = np.load(file_path)
        c, h, w = X.shape
        if X.shape == DIMS:
            X = np.moveaxis(X, 0, -1)
            
            X = np.reshape(X, (-1, DIMS[0]))
            return X, c, h, w
    except Exception:
        # Ignora arquivos corrompidos ou com erro de leitura
        return None


def split_data(data_list, group=4):
    output_data = data_list
    step = group // 2
    for i in range(step):
        split_data = []
        for data in output_data:
            n, c = data.shape
            data_s1 = data[:, :c // 2]
            data_s2 = data[:, c // 2:]
            split_data.append(data_s1)
            split_data.append(data_s2)
        output_data = split_data
    return output_data


def applyGWPCA(X, nc=32, group=4, whiten=True):
    h, w, c = X.shape
    X = np.reshape(X, (-1, c))
    X = (X - X.min()) / (np.max(X) - np.min(X))

    X_split = split_data([X], group)
    pca_data_list = []
    for i, x in enumerate(X_split):
        pca = PCA(n_components=nc // group, whiten=whiten, random_state=42)
        pca_data = pca.fit_transform(x)
        pca_data_list.append(pca_data)

    out = np.concatenate(pca_data_list, axis=-1)
    out = np.reshape(out, (h, w, -1))
    return out





In [10]:
X = np.load(npy_files[0])
# colocar o canal como a última dimensão
X = np.moveaxis(X, 0, -1)

h, w, c = X.shape
whiten = True
group = 4
nc = 32

X = np.reshape(X, (-1, DIMS[0]))


X_split = split_data([X], group)
pca_data_list = []
for i, x in enumerate(X_split):
    pca = PCA(n_components=nc // group, whiten=whiten, random_state=42)
    pca_data = pca.fit_transform(x)
    pca_data_list.append(pca_data)

out = np.concatenate(pca_data_list, axis=-1)
out = np.reshape(out, (h, w, -1))


In [11]:
pca.components_.shape

(8, 51)

In [12]:
pca_data_list[0].shape

(16384, 8)

In [18]:
split_data([process_single_file(npy_files[0])])[0].shape

(16384, 50)

In [14]:
# incremental pca

# create the four ipca
standardscaler = [StandardScaler() for _ in range(NUM_GROUPS)]
ipcas = [IncrementalPCA(n_components=NC_PER_GROUP, whiten=True) for _ in range(NUM_GROUPS)]


for npy_file in tqdm(npy_files, desc="Loading and processing files"):
    pixel_matrix = process_single_file(npy_file)
    if pixel_matrix is not None:
        # Dividir os dados em grupos
        grouped_data = split_data([pixel_matrix], group=NUM_GROUPS)
        for i, group_data in enumerate(grouped_data):
            # Ajustar o IPCA incrementalmente
            ipcas[i].partial_fit(group_data)


Loading and processing files: 100%|██████████| 11483/11483 [16:46<00:00, 11.41it/s]


In [22]:
# apply the pca tranform in all data

name_file = 0

# add 
file_mapping = []

for npy_file in tqdm(npy_files, desc="Transforming and saving files"):
    name_file += 1
    pixel_matrix, c, h, w = process_single_file(npy_file)
    if pixel_matrix is not None:
        # Dividir os dados em grupos
        grouped_data = split_data([pixel_matrix], group=NUM_GROUPS)
        pca_data_list = []
        for i, group_data in enumerate(grouped_data):
            # Transformar os dados usando o IPCA ajustado
            pca_data = ipcas[i].transform(group_data)
            pca_data_list.append(pca_data)



        # Concatenar os dados PCA de todos os grupos
        out = np.concatenate(pca_data_list, axis=-1)
        out = np.reshape(out, (h, w, -1))

        file_mapping.append(
            {
                "original_file": npy_file,
                "pca_file": os.path.join(out_directory, f"pca_{name_file:05d}_DATA.npy"),
                "name_file": name_file

            }
        )
        # Salvar o resultado em um arquivo .npy
        output_file = os.path.join(out_directory, f"pca_{name_file:05d}_DATA.npy")
        np.save(output_file, out)


Transforming and saving files: 100%|██████████| 11483/11483 [13:52<00:00, 13.79it/s]


In [23]:
# file mapping to csv

import pandas as pd
df_mapping = pd.DataFrame(file_mapping)
df_mapping.to_csv(os.path.join(out_directory, "file_mapping.csv"), index=False)

In [24]:
df_mapping

,original_file,pca_file,name_file
0,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_00001_DATA.npy,1
1,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_00002_DATA.npy,2
2,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_00003_DATA.npy,3
3,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_00004_DATA.npy,4
4,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_00005_DATA.npy,5
...,...,...,...
11478,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_11479_DATA.npy,11479
11479,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_11480_DATA.npy,11480
11480,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_11481_DATA.npy,11481
11481,./../../hyspecnet-11k/hyspecnet-11k/patches/EN...,./../../hyspecnet11k-PCA/pca_11482_DATA.npy,11482
